In [1]:
import sys
import matplotlib.pyplot as plt
from glob import glob
import xarray as xr
import rasterio as rio
import rioxarray
import os
import numpy as np
import pandas as pd

In [2]:
#!{sys.executable} -m pip install phase_o_matic

In [3]:
from phase_o_matic import presto_phase_delay

In [4]:
orbit = 'DT56'
year_list = ['2018', '2019', '2020', '2021', '2022']
frame = 'frame_3'

In [5]:
# load in single igram and other data 
def hyp3_to_xarray_single(path):
    '''
    Reads unwrapped phase, coherence, and DEM into xarray dataset from single hyp3 folder 
    '''
    # globs for data to load
    unw_phase_path = glob(f'{path}/*unw_phase.tif')[0]
    dem_path = glob(f'{path}/*dem.tif')[0]
    inc_path = glob(f'{path}/*inc_map.tif')[0]

    # list granules for coordinate
    granule = os.path.split(unw_phase_path)[-1][0:-14]

    # read unw_phase into data array and assign coordinates
    da = xr.open_dataset(unw_phase_path)
    da = da.assign_coords({'granule':('granule', [granule])})
    
    # concatenate into dataset and rename variable
    ds = da.rename({'band_data': 'unw_phase'})

    #open coherence and dem into datasets
    inc_ds = xr.open_dataset(inc_path)
    dem_ds = xr.open_dataset(dem_path)

    # add coherence and dem to unw_phase dataset
    ds['inc_angle'] = (('band', 'y', 'x'), inc_ds.band_data.values)
    ds['elevation'] = (('band', 'y', 'x'), dem_ds.band_data.values)

    # remove band coordinate
    ds = ds.squeeze()

    return ds

In [6]:
def calc_delay(ds, work_dir):
    """
    calculate ERA5 delays with phase-o-matic
    """
    time1 = pd.to_datetime(ds.granule.item()[5:20], utc=True)
    time2 = pd.to_datetime(ds.granule.item()[21:36], utc=True)

    t1 = presto_phase_delay(date=time1, dem=ds.elevation, inc=ds.inc_angle, work_dir=work_dir, wavelength=0.55465763)
    t2 = presto_phase_delay(date=time2, dem=ds.elevation, inc=ds.inc_angle, work_dir=work_dir, wavelength=0.55465763)

    delay_change = t2.isel(time = 0)['delay'] - t1.isel(time = 0)['delay']
    
    ds['unw_phase_era5'] = (('y', 'x'), delay_change.values) 

    return ds

In [7]:
def calc_save_delays(hyp3_path, work_dir):
    hyp3_list = os.listdir(hyp3_path)
    for granule in hyp3_list:
        print(f'working on {granule}')
        granule_path = f'{hyp3_path}/{granule}'
        
        print('opening dataset')
        ds = hyp3_to_xarray_single(granule_path)

        ds_wgs = ds.rio.reproject(4326, nodata=np.nan)
        ds_wgs = ds_wgs.rename({'y':'latitude', 'x':'longitude'})
        print('calculating era5 delays')
        ds_wgs = calc_delay(ds_wgs, work_dir)
        ds = ds_wgs.reproject_match(ds)
        fn = f'{granule}_unw_phase_era5.tif'
        
        print(f'saving {fn}')
        ds.unw_phase_era5.rio.to_raster(f'{granule_path}/{fn}.tif')
    print('done')

In [8]:
def multiyear_era5(orbit, year_list, frame_list):
    # hardcoded paths for now 
    home_path = '/mnt/c/Users/qbren/Desktop/taco/projects/indennt/proc/data'
    for frame in frame_list:
        print(f'working on {frame}')
        for year in year_list:
            print(f'working on {year}')
            data_path = f'{home_path}/hyp3/{orbit}/{frame}/{year}'
            
            calc_save_delays(data_path, '/tmp')

In [ ]:
multiyear_era5(orbit, [year], [frame])

working on frame_3
working on 2017
working on S1AA_20170607T130955_20170619T130955_VVP012_INT40_G_ueF_3D12
opening dataset
calculating era5 delays
